In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

GBM Function

In [10]:
def simulate_gbm(s_0, mu, sigma, T=1, dt=1/252, n_sims=100):
    n_steps = int(T / dt)
    shocks = np.random.standard_normal((n_steps, n_sims))

    paths = np.zeros((n_steps + 1, n_sims))
    paths[0] = s_0

    for t in range(1, n_steps + 1):
        paths[t] = paths[t - 1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * shocks[t - 1]
        )

    time_index = np.linspace(0, T, n_steps + 1)
    paths_df = pd.DataFrame(paths, index=time_index)
    paths_df.index.name = "Time (Years)"
    return paths_df

CPPI Function

In [11]:
def run_cppi(risky_r, safe_r=0.03, m=3, start_value=1000, floor_pct=0.80):
    account_value = start_value
    floor_value = start_value * floor_pct

    n_steps = len(risky_r)
    account_history = [start_value]
    floor_history = [floor_value]
    cushion_history = [start_value - floor_value]
    risky_alloc_history = []

    sim_index = risky_r.index

    for t in range(n_steps):
        cushion = account_value - floor_value
        cushion_history.append(cushion)

        risky_alloc = np.clip(m * cushion, 0, account_value)
        safe_alloc = account_value - risky_alloc
        risky_alloc_history.append(risky_alloc)

        account_value = (risky_alloc * (1 + risky_r.iloc[t])) + \
                        (safe_alloc * (1 + safe_r))

        floor_value *= (1 + safe_r)

        account_history.append(account_value)
        floor_history.append(floor_value)

    cushion = account_value - floor_value
    risky_alloc = np.clip(m * cushion, 0, account_value)
    risky_alloc_history.append(risky_alloc)

    cppi_results = pd.DataFrame({
        "AccountValue": account_history,
        "FloorValue": floor_history,
        "RiskyAllocation": risky_alloc_history
    }, index=sim_index.insert(0, 0))

    return cppi_results

Run Simulations

In [12]:
S0 = 100
MU = 0.07
SIGMA = 0.15
T_YEARS = 3
DT = 1/252
N_SIMS = 50

print("Running GBM Simulation...")
gbm_paths = simulate_gbm(S0, MU, SIGMA, T_YEARS, DT, N_SIMS)
print("GBM Simulation Complete.")

risky_path_prices = gbm_paths[0]
risky_r = risky_path_prices.pct_change().dropna()

SAFE_R_ANNUAL = 0.03
SAFE_R_DAILY = (1 + SAFE_R_ANNUAL)**DT - 1
MULTIPLIER = 3
START_VALUE = 1000
FLOOR_PCT = 0.80

print("Running CPPI Simulation...")
cppi_results = run_cppi(risky_r, safe_r=SAFE_R_DAILY, m=MULTIPLIER,
                        start_value=START_VALUE, floor_pct=FLOOR_PCT)
print("CPPI Simulation Complete.")

Running GBM Simulation...
GBM Simulation Complete.
Running CPPI Simulation...
CPPI Simulation Complete.


GBM Plot

In [13]:
fig_gbm = go.Figure()
for col in gbm_paths.columns:
    fig_gbm.add_trace(go.Scatter(x=gbm_paths.index, y=gbm_paths[col],
                             mode='lines', line=dict(width=1, color='blue'),
                             opacity=0.3, showlegend=False))

fig_gbm.add_trace(go.Scatter(x=gbm_paths.index, y=gbm_paths[0],
                         mode='lines', line=dict(width=2, color='red'),
                         name='Path used for CPPI'))

fig_gbm.update_layout(
    title=f"Monte Carlo Simulation of GBM ({N_SIMS} Paths)",
    xaxis_title="Time (Years)",
    yaxis_title="Asset Price ($)",
    hovermode="x unified"
)
fig_gbm.show()

CPPI Plot

In [14]:
scaled_risky_asset = risky_path_prices * (START_VALUE / S0)

fig_cppi = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Portfolio & Risky Asset Value", "Risky Allocation ($)")
)

fig_cppi.add_trace(
    go.Scatter(x=cppi_results.index, y=cppi_results['AccountValue'],
               name="Account Value", line=dict(color='blue', width=2)),
    row=1, col=1
)
fig_cppi.add_trace(
    go.Scatter(x=cppi_results.index, y=cppi_results['FloorValue'],
               name="Floor Value", line=dict(color='red', width=2, dash='dot')),
    row=1, col=1
)
fig_cppi.add_trace(
    go.Scatter(x=scaled_risky_asset.index, y=scaled_risky_asset,
               name="Risky Asset (Scaled)", line=dict(color='grey', width=1, dash='dash')),
    row=1, col=1
)

fig_cppi.add_trace(
    go.Scatter(x=cppi_results.index, y=cppi_results['RiskyAllocation'],
               name="Risky Allocation ($)", line=dict(color='green'), fill='tozeroy'),
    row=2, col=1
)

fig_cppi.update_layout(
    title_text=f"CPPI Strategy Simulation (Multiplier={MULTIPLIER}, Floor={FLOOR_PCT*100}%)",
    hovermode="x unified",
    xaxis_title="Time (Days)",
    xaxis2_title="Time (Days)",
    yaxis_title="Value ($)",
    yaxis2_title="Allocation ($)",
    legend_tracegroupgap=180
)
fig_cppi.show()

#CONCLUSION
This lab successfully demonstrated the implementation and analysis of both Geometric Brownian Motion (GBM) and Constant Proportion Portfolio Insurance (CPPI) strategies. The Monte Carlo simulation produced 50 distinct, random asset price paths, effectively modeling the stochastic nature of financial markets as described by GBM. One of these paths was then used as the underlying risky asset to test the CPPI strategy.

The CPPI simulation results clearly illustrate the strategy's core function. The portfolio's "Account Value" successfully remained above the rising "Floor Value" for the entire duration, achieving its primary objective of downside protection. The dynamic nature of CPPI was evident in the "Risky Allocation" plot, which showed the investment in the risky asset increasing as the cushion (the gap between account value and floor) grew, and automatically decreasing as the cushion shrank. This experiment confirms that CPPI is an effective dynamic strategy for insuring a portfolio's principal while still allowing for participation in market gains.